In [7]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist=input_data.read_data_sets("/tmp/data/",one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [8]:
n_input=784
n_output=10

weights={
    'wc1':tf.Variable(tf.random_normal([3,3,1,32])),
    'wc2':tf.Variable(tf.random_normal([3,3,32,64])),
    'wd1':tf.Variable(tf.random_normal([7*7*64,50])),
    'wd2':tf.Variable(tf.random_normal([50,n_output]))
}
biases={
    'bc1':tf.Variable(tf.random_normal([32])),
    'bc2':tf.Variable(tf.random_normal([64])),
    'bd1':tf.Variable(tf.random_normal([50])),
    'bd2':tf.Variable(tf.random_normal([n_output]))
}

In [9]:
def conv_basic(X,W,b,keepratic):
    X=tf.reshape(X,shape=[-1,28,28,1])  #-1为让tensorflow自己推段 
    conv1=tf.nn.conv2d(X,W['wc1'],strides=[1,1,1,1],padding='SAME') 
    #strides的四维分别是batch_size,长，宽，深度，一般在长和宽上移动就可以了
    conv1=tf.nn.relu(tf.nn.bias_add(conv1,b['bc1']))
    
    pool1=tf.nn.max_pool(conv1,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
    #pool1_dr1=tf.nn.dropout(pool1,keepratic)#keepratic为dropout的比例
    
    #ksize选择的windows,有4个维度，分别是batch_size,长，宽，深度
    conv2=tf.nn.conv2d(pool1,W['wc2'],strides=[1,1,1,1],padding='SAME')
    conv2=tf.nn.relu(tf.nn.bias_add(conv2,b['bc2']))
    
    pool2=tf.nn.max_pool(conv2,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
    #pool2_dr2=tf.nn.dropout(pool2,keepratic)
    
    #接下来为全连接层
    dense1=tf.reshape(pool2,[-1,W['wd1'].get_shape().as_list()[0]])
    fc1=tf.nn.relu(tf.add(tf.matmul(dense1,W['wd1']),b['bd1']))
    #fc_dr1=tf.nn.dropout(fc1,keepratic)
    
    out=tf.add(tf.matmul(fc1,W['wd2']),b['bd2'])
    
    return out
print("CNN READY!")

CNN READY!


In [10]:
X=tf.placeholder(tf.float32,shape=[None,784],name='X_input')
Y=tf.placeholder(tf.float32,shape=[None,10],name='Y_input')
keepratic=tf.placeholder(tf.float32)

pred=conv_basic(X,weights,biases,keepratic)
cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred,labels=Y))
optm=tf.train.AdamOptimizer(0.001).minimize(cost)
corr=tf.equal(tf.argmax(pred,1),tf.argmax(Y,1))
acrr=tf.reduce_mean(tf.cast(corr,tf.float32))

In [11]:
batch_size=128
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    for epoch in range(10):
        num=int(mnist.train.num_examples/batch_size)
        avg_cost=0
        for i in range(num):
            batch_xs,batch_ys=mnist.train.next_batch(batch_size)
            _,l=sess.run([optm,cost],feed_dict={X:batch_xs,Y:batch_ys,keepratic:0.7})
            avg_cost+=l/num
        '''corr_train=tf.equal(tf.argmax(pred,1),tf.argmax(Y,1))
        acrr_train=tf.reduce_mean(tf.cast(corr_train,tf.float32))
        train_acrr=sess.run(acrr_train,feed_dict={X:batch_xs,Y:batch_ys,keepratic:1.0})
        '''   
        corr_test=tf.equal(tf.argmax(pred,1),tf.argmax(Y,1))
        acrr_test=tf.reduce_mean(tf.cast(corr_test,tf.float32))
        #test_acrr=sess.run(acrr_test,feed_dict={X:mnist.test.images,Y:mnist.test.labels,keepratic:1.0})
        print("Epoch:",epoch,"loss:",avg_cost,"测试集准确率：",acrr_test.eval({X:mnist.test.images,Y:mnist.test.labels,keepratic:1.0}))

Epoch: 0 loss: 397.105650584 测试集准确率： 0.8417
Epoch: 1 loss: 59.9256066987 测试集准确率： 0.8942
Epoch: 2 loss: 36.6149689912 测试集准确率： 0.9138
Epoch: 3 loss: 25.570972292 测试集准确率： 0.9243
Epoch: 4 loss: 19.1384784742 测试集准确率： 0.9319
Epoch: 5 loss: 15.0406619302 测试集准确率： 0.9385
Epoch: 6 loss: 12.0706775692 测试集准确率： 0.9464
Epoch: 7 loss: 10.0569074074 测试集准确率： 0.946
Epoch: 8 loss: 8.42617868241 测试集准确率： 0.9524
Epoch: 9 loss: 6.92490983698 测试集准确率： 0.9533
